In [10]:
import pygame
from pygame.locals import DOUBLEBUF, OPENGL
from OpenGL.GL import *
from OpenGL.GLU import *
import numpy as np
import ctypes
import time
import sys
import os

class FluidSimulation:
    """High-performance fluid simulation using CUDA Fortran and OpenGL visualization."""
    
    def __init__(self, resolution=128, lib_path="./fluid_dynamics.so"):
        """Initialize the fluid simulation with the given resolution."""
        self.resolution = resolution
        self.window_size = (1024, 768)
        
        # Initialization flag for cleanup management
        self.initialized = False
        
        # Load the CUDA Fortran library
        try:
            self.lib = ctypes.CDLL(lib_path)
            print(f"Loaded library: {lib_path}")
        except Exception as e:
            print(f"Error loading library: {e}")
            raise
        
        # Set up function signatures
        self._setup_functions()
        
        # Initialize CUDA resources and simulation
        try:
            self.lib.initialize_fluid(ctypes.c_int(self.resolution))
            self.initialized = True
        except Exception as e:
            print(f"Error initializing fluid: {e}")
            raise
        
        # Create buffer for getting density data
        self.density_buffer = np.zeros((self.resolution, self.resolution), dtype=np.float64)
        
        # Initialize texture ID to None - will be created in setup_opengl
        self.texture_id = None
        
        # Performance tracking
        self.fps = 0
        
    def _setup_functions(self):
        """Set up function signatures for the CUDA Fortran library."""
        # Core simulation functions
        self.lib.initialize_fluid.argtypes = [ctypes.c_int]
        self.lib.cleanup_fluid.argtypes = []
        self.lib.step_fluid.argtypes = []
        self.lib.reset_fluid.argtypes = []
        
        # Interaction functions
        self.lib.add_density.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_double]
        self.lib.add_velocity.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_double, ctypes.c_double]
        
        # Data access functions
        self.lib.get_density.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64, shape=(self.resolution, self.resolution))]
        
        # Try to set up optional debug function if available
        try:
            self.lib.print_density_info.argtypes = []
        except:
            print("Warning: print_density_info function not available")
        
    def setup_opengl(self):
        """Set up OpenGL for rendering the fluid simulation."""
        # Set a dark background color
        glClearColor(0.1, 0.1, 0.1, 1.0)
        
        # Set up projection matrix
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluOrtho2D(0, 1, 0, 1)  # 2D orthographic projection
        
        # Set up modelview matrix
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Enable texturing and blending
        glEnable(GL_TEXTURE_2D)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        
        # Delete any existing texture first to prevent leaks
        if self.texture_id is not None:
            try:
                glDeleteTextures([self.texture_id])
            except:
                pass
        
        # Create texture for fluid display
        self.texture_id = glGenTextures(1)
        glBindTexture(GL_TEXTURE_2D, self.texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
        
        # Initialize with a blank texture
        blank_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
        blank_data[:,:,3] = 255  # Full alpha
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0,
                    GL_RGBA, GL_UNSIGNED_BYTE, blank_data)
        
        print(f"OpenGL initialized with texture ID: {self.texture_id}")
        
        # Check for OpenGL errors
        error = glGetError()
        if error != GL_NO_ERROR:
            print(f"OpenGL error during setup: {error}")
    
    def draw_test_pattern(self):
        """Draw a simple test pattern to verify OpenGL rendering."""
        if not self.initialized:
            return
            
        # Create a checkerboard pattern
        test_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
        
        # Make a more obvious pattern with multiple colors
        square_size = max(4, self.resolution // 16)
        for i in range(self.resolution):
            for j in range(self.resolution):
                pattern_type = ((i // square_size) + (j // square_size)) % 4
                
                if pattern_type == 0:
                    test_data[i, j, 0] = 255  # Red
                    test_data[i, j, 1] = 0
                    test_data[i, j, 2] = 0
                elif pattern_type == 1:
                    test_data[i, j, 0] = 0
                    test_data[i, j, 1] = 255  # Green
                    test_data[i, j, 2] = 0
                elif pattern_type == 2:
                    test_data[i, j, 0] = 0
                    test_data[i, j, 1] = 0
                    test_data[i, j, 2] = 255  # Blue
                else:
                    test_data[i, j, 0] = 255  # Yellow
                    test_data[i, j, 1] = 255
                    test_data[i, j, 2] = 0
                    
                test_data[i, j, 3] = 255  # Alpha
        
        # Ensure we have a valid texture ID
        if self.texture_id is None:
            self.texture_id = glGenTextures(1)
        
        # Bind and update the texture
        glBindTexture(GL_TEXTURE_2D, self.texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0, 
                    GL_RGBA, GL_UNSIGNED_BYTE, test_data)
        
        # Check for OpenGL errors
        error = glGetError()
        if error != GL_NO_ERROR:
            print(f"OpenGL error when creating test pattern: {error}")
        else:
            print("Test pattern texture created successfully")
    
    def update_texture(self):
        """Update the fluid texture with the current density field."""
        if not self.initialized:
            return
            
        try:
            # Get density data from the simulation
            self.lib.get_density(self.density_buffer)
            
            # Create texture data
            texture_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
            
            # Set a baseline dark blue background color
            texture_data[:,:,2] = 30  # Low blue background
            texture_data[:,:,3] = 255  # Full alpha
            
            # Normalize the density values
            # Use 95th percentile to avoid extreme outliers
            p95 = np.percentile(self.density_buffer, 95)
            norm_factor = 1.0 / max(p95, 0.1)
            
            # Map density values to colors
            for i in range(self.resolution):
                for j in range(self.resolution):
                    value = self.density_buffer[i, j] * norm_factor
                    value = min(value, 1.0)  # Clamp to [0,1]
                    
                    if value > 0.01:  # Threshold to avoid rendering very small values
                        # Color gradient from blue to cyan to yellow to red
                        if value < 0.25:
                            # Blue to cyan
                            ratio = value / 0.25
                            texture_data[i, j, 0] = 0  # No red
                            texture_data[i, j, 1] = int(255 * ratio)  # Increasing green
                            texture_data[i, j, 2] = 255  # Full blue
                        elif value < 0.5:
                            # Cyan to green
                            ratio = (value - 0.25) / 0.25
                            texture_data[i, j, 0] = 0  # No red
                            texture_data[i, j, 1] = 255  # Full green
                            texture_data[i, j, 2] = int(255 * (1 - ratio))  # Decreasing blue
                        elif value < 0.75:
                            # Green to yellow
                            ratio = (value - 0.5) / 0.25
                            texture_data[i, j, 0] = int(255 * ratio)  # Increasing red
                            texture_data[i, j, 1] = 255  # Full green
                            texture_data[i, j, 2] = 0  # No blue
                        else:
                            # Yellow to red
                            ratio = (value - 0.75) / 0.25
                            texture_data[i, j, 0] = 255  # Full red
                            texture_data[i, j, 1] = int(255 * (1 - ratio))  # Decreasing green
                            texture_data[i, j, 2] = 0  # No blue
            
            # Make sure we have a valid texture
            if self.texture_id is None:
                self.texture_id = glGenTextures(1)
                glBindTexture(GL_TEXTURE_2D, self.texture_id)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
            
            # Update the texture
            glBindTexture(GL_TEXTURE_2D, self.texture_id)
            glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0, 
                        GL_RGBA, GL_UNSIGNED_BYTE, texture_data)
        except Exception as e:
            print(f"Error updating texture: {e}")
    
    def step(self):
        """Advance the simulation by one time step."""
        if self.initialized:
            try:
                self.lib.step_fluid()
            except Exception as e:
                print(f"Error stepping fluid simulation: {e}")
    
    def add_density(self, x, y, amount=1.0):
        """Add density at the specified position."""
        if not self.initialized:
            return
            
        try:
            # Convert from normalized [0,1] coordinates to grid coordinates
            grid_x = int(x * self.resolution)
            grid_y = int(y * self.resolution)
            
            # Clamp to valid range
            grid_x = max(1, min(grid_x, self.resolution-2))
            grid_y = max(1, min(grid_y, self.resolution-2))
            
            # Add density with high amount to make it visible
            self.lib.add_density(ctypes.c_int(grid_x), ctypes.c_int(grid_y), ctypes.c_double(amount * 100.0))
            
            # Also add density to surrounding cells for a more visible effect
            if grid_x > 1:
                self.lib.add_density(ctypes.c_int(grid_x-1), ctypes.c_int(grid_y), ctypes.c_double(amount * 50.0))
            
            if grid_x < self.resolution-2:
                self.lib.add_density(ctypes.c_int(grid_x+1), ctypes.c_int(grid_y), ctypes.c_double(amount * 50.0))
            
            if grid_y > 1:
                self.lib.add_density(ctypes.c_int(grid_x), ctypes.c_int(grid_y-1), ctypes.c_double(amount * 50.0))
            
            if grid_y < self.resolution-2:
                self.lib.add_density(ctypes.c_int(grid_x), ctypes.c_int(grid_y+1), ctypes.c_double(amount * 50.0))
        except Exception as e:
            print(f"Error adding density: {e}")
    
    def add_velocity(self, x, y, vx, vy):
        """Add velocity at the specified position."""
        if not self.initialized:
            return
            
        try:
            # Convert from normalized [0,1] coordinates to grid coordinates
            grid_x = int(x * self.resolution)
            grid_y = int(y * self.resolution)
            
            # Clamp to valid range
            grid_x = max(1, min(grid_x, self.resolution-2))
            grid_y = max(1, min(grid_y, self.resolution-2))
            
            # Scale up velocity for more visible effect
            self.lib.add_velocity(ctypes.c_int(grid_x), ctypes.c_int(grid_y), 
                                ctypes.c_double(vx * 10.0), ctypes.c_double(vy * 10.0))
        except Exception as e:
            print(f"Error adding velocity: {e}")
    
    def reset(self):
        """Reset the simulation to its initial state."""
        if self.initialized:
            try:
                self.lib.reset_fluid()
            except Exception as e:
                print(f"Error resetting fluid simulation: {e}")
    
    def change_resolution(self, new_resolution):
        """Change the simulation resolution."""
        if not self.initialized or new_resolution == self.resolution:
            return
            
        try:
            # Clean up current simulation
            self.lib.cleanup_fluid()
            self.initialized = False
            
            # Update resolution
            self.resolution = new_resolution
            
            # Reinitialize
            self._setup_functions()  # Update function signatures for new resolution
            self.lib.initialize_fluid(ctypes.c_int(self.resolution))
            self.initialized = True
            
            # Reset density buffer
            self.density_buffer = np.zeros((self.resolution, self.resolution), dtype=np.float64)
            
            # Reset texture ID so it will be recreated with the new resolution
            if self.texture_id is not None:
                try:
                    glDeleteTextures([self.texture_id])
                except:
                    pass
                self.texture_id = None
            
            print(f"Resolution changed to {self.resolution}x{self.resolution}")
        except Exception as e:
            print(f"Error changing resolution: {e}")
    
    def cleanup(self):
        """Explicit cleanup method to be called when shutting down."""
        if self.initialized:
            try:
                # First clean up OpenGL resources
                if self.texture_id is not None:
                    try:
                        glDeleteTextures([self.texture_id])
                        self.texture_id = None
                    except Exception as e:
                        print(f"Error deleting texture: {e}")
                
                # Then clean up CUDA resources
                self.lib.cleanup_fluid()
                self.initialized = False
                print("Fluid simulation resources cleaned up")
            except Exception as e:
                print(f"Error during cleanup: {e}")
                self.initialized = False
    
    def __del__(self):
        """Clean up resources when the object is destroyed."""
        self.cleanup()


class FluidSimulationApp:
    """Main application for the fluid simulation."""
    
    def __init__(self, initial_resolution=128, use_fullscreen=True):
        """Initialize the application with the given resolution."""
        # Initialize PyGame
        pygame.init()
        
        # Set default window size for all modes
        self.window_size = (1024, 768)
        
        # Create display with desired mode
        if use_fullscreen:
            display_info = pygame.display.Info()
            self.window_size = (display_info.current_w, display_info.current_h)
            # self.display = pygame.display.set_mode(self.window_size, DOUBLEBUF | OPENGL | pygame.FULLSCREEN)
            self.display = pygame.display.set_mode((0,0), DOUBLEBUF | OPENGL | pygame.FULLSCREEN)            
            self.fullscreen = True
        else:
            self.display = pygame.display.set_mode(self.window_size, DOUBLEBUF | OPENGL)
            self.fullscreen = False
        
        # Get actual window size
        self.window_size = pygame.display.get_surface().get_size()
        pygame.display.set_caption("Fluid Dynamics Simulation")
        
        print(f"Window initialized: size={self.window_size}, fullscreen={self.fullscreen}")
        
        # Store resolution
        self.resolution = initial_resolution
        
        # Create the fluid simulation - Initialize only once!
        self.fluid = FluidSimulation(initial_resolution)
        
        # Set up OpenGL
        self.fluid.setup_opengl()
        
        # Make sure our viewport is set up correctly
        self.reset_opengl_viewport()
        
        # Mouse tracking
        self.prev_mouse_pos = None
        self.mouse_down = False
        
        # Control flags
        self.paused = False
        
        print("Controls: R=Reset, Space=Pause, ESC=Exit")
    
    def reset_opengl_viewport(self):
        """Reset the OpenGL viewport and projection to fit the current window."""
        # Ensure we have accurate window dimensions
        self.window_size = pygame.display.get_surface().get_size()
        # Set viewport to cover the whole window
        #glViewport(0, 0, self.window_size[0], self.window_size[1])

        # Set viewport to be slightly larger than the window
        extra_pixels = 100  # Add a few pixels on each side to avoid bevelling on edges **********************************
        glViewport(-extra_pixels, -extra_pixels, 
                self.window_size[0] + 2*extra_pixels, 
                self.window_size[1] + 2*extra_pixels)
        
        # Reset projection matrix for 2D rendering
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluOrtho2D(0, 1, 0, 1)  # Map to [0,1] coordinate space
        
        # Reset modelview matrix
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Make sure texturing and blending are enabled
        glEnable(GL_TEXTURE_2D)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
    
    def toggle_fullscreen(self):
        """Toggle fullscreen mode."""
        # Toggle fullscreen state
        self.fullscreen = not self.fullscreen
        
        # Save current resolution before switching
        current_resolution = self.fluid.resolution
        
        # Properly clean up current OpenGL context and resources
        glFinish()  # Make sure all OpenGL commands are done
        
        # Clean up texture resources
        if self.fluid.texture_id is not None:
            try:
                glDeleteTextures([self.fluid.texture_id])
                self.fluid.texture_id = None
            except Exception as e:
                print(f"Error cleaning up texture: {e}")
        
        # Create new display with desired mode
        if self.fullscreen:
            # Get display info for native resolution
            display_info = pygame.display.Info()
            new_size = (display_info.current_w, display_info.current_h)
            self.display = pygame.display.set_mode(new_size, DOUBLEBUF | OPENGL | pygame.FULLSCREEN)
        else:
            # Return to windowed mode with standard size
            self.display = pygame.display.set_mode((1024, 768), DOUBLEBUF | OPENGL)
        
        # Update window size
        self.window_size = pygame.display.get_surface().get_size()
        print(f"Toggled to {'fullscreen' if self.fullscreen else 'windowed'} mode, size: {self.window_size}")
                
        # Recreate the fluid texture
        self.fluid.setup_opengl()
        
        # Reset viewport
        self.reset_opengl_viewport()
    
    def draw_fluid_texture(self):
        """Draw the fluid simulation texture to fill the entire screen."""
        # Ensure we have a valid texture
        if self.fluid.texture_id is None:
            return
            
        try:
            # Set up for drawing
            glMatrixMode(GL_MODELVIEW)
            glLoadIdentity()
            
            # Bind the fluid simulation texture
            glEnable(GL_TEXTURE_2D)
            glBindTexture(GL_TEXTURE_2D, self.fluid.texture_id)
            
            # Draw a quad that covers the entire screen in the [0,1] coordinate space
            glColor4f(1.0, 1.0, 1.0, 1.0)  # Full color/alpha
            glBegin(GL_QUADS)
            # These coordinates map to the entire viewport in the [0,1] space
            glTexCoord2f(0.0, 0.0); glVertex2f(0.0, 0.0)  # Bottom-left
            glTexCoord2f(1.0, 0.0); glVertex2f(1.0, 0.0)  # Bottom-right
            glTexCoord2f(1.0, 1.0); glVertex2f(1.0, 1.0)  # Top-right
            glTexCoord2f(0.0, 1.0); glVertex2f(0.0, 1.0)  # Top-left
            glEnd()
            
            # Disable texturing when done
            glDisable(GL_TEXTURE_2D)
        except Exception as e:
            print(f"Error drawing fluid texture: {e}")
    
    def run(self):
        """Run the main application loop."""
        running = True
        clock = pygame.time.Clock()
        
        # Visualization mode - start with simulation
        show_simulation = True
        
        print("Starting fluid simulation. Press ESC to exit.")
        
        try:
            while running:
                # Handle events
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                        
                    elif event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            running = False
                        elif event.key == pygame.K_r:
                            self.fluid.reset()
                            print("Simulation reset")
                        elif event.key == pygame.K_SPACE:
                            self.paused = not self.paused
                            print(f"Simulation {'paused' if self.paused else 'resumed'}")
                        elif event.key == pygame.K_t:
                            # Toggle between test pattern and simulation
                            show_simulation = not show_simulation
                            if not show_simulation:
                                self.fluid.draw_test_pattern()
                                print("Switched to test pattern")
                            else:
                                print("Switched to simulation")
                        elif event.key == pygame.K_UP:
                            # Increase resolution
                            new_res = min(self.fluid.resolution * 2, 512)
                            self.fluid.change_resolution(new_res)
                        elif event.key == pygame.K_DOWN:
                            # Decrease resolution
                            new_res = max(self.fluid.resolution // 2, 32)
                            self.fluid.change_resolution(new_res)
                        #elif event.key == pygame.K_f:
                            # Toggle fullscreen
                            # self.toggle_fullscreen()
                            
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        if event.button == 1:  # Left mouse button
                            self.mouse_down = True
                            self.prev_mouse_pos = pygame.mouse.get_pos()
                            
                            # Add initial density at the exact mouse position
                            if show_simulation:
                                x, y = self.prev_mouse_pos
                                
                                # Fix the coordinate mapping:
                                # Since we're seeing up→right and clockwise→counter-clockwise,
                                # we need to swap x and y AND flip one of them
                                # Try this mapping: mouse_x → fluid_y, mouse_y → (1-fluid_x)
                                nx = 1.0 - y / self.window_size[1]  # Inverted y becomes x
                                ny = x / self.window_size[0]        # x becomes y
                                
                                self.fluid.add_density(nx, ny, 2.0)
                                print(f"Added density at {nx:.2f}, {ny:.2f}")
                            
                    elif event.type == pygame.MOUSEBUTTONUP:
                        if event.button == 1:  # Left mouse button
                            self.mouse_down = False
                            self.prev_mouse_pos = None
                
                # Handle mouse motion
                if self.mouse_down and show_simulation:
                    x, y = pygame.mouse.get_pos()
                    
                    # Fix the coordinate mapping (same as in mouse click handling)
                    nx = 1.0 - y / self.window_size[1]  # Inverted y becomes x
                    ny = x / self.window_size[0]        # x becomes y
                    
                    self.fluid.add_density(nx, ny, 1.0)
                    
                    # Add velocity if mouse has moved
                    if self.prev_mouse_pos is not None:
                        prev_x, prev_y = self.prev_mouse_pos
                        
                        # Calculate velocity with the same coordinate transformation
                        dx = x - prev_x
                        dy = y - prev_y
                        
                        # Transform the velocity vector to match our coordinate transformation
                        vx = -dy / self.window_size[1] * 5.0  # negative dy becomes vx
                        vy = dx / self.window_size[0] * 5.0   # dx becomes vy
                        
                        # Add velocity
                        self.fluid.add_velocity(nx, ny, vx, vy)
                    
                    # Update previous position
                    self.prev_mouse_pos = (x, y)
                
                # Step the simulation if not paused and showing simulation
                if not self.paused and show_simulation:
                    self.fluid.step()
                    self.fluid.update_texture()
                
                # Clear the screen
                glClear(GL_COLOR_BUFFER_BIT)
                
                # Draw the current texture (test pattern or simulation)
                self.draw_fluid_texture()
                
                # Text rendering with proper cleanup
                text_texture = None
                try:
                    # Create and setup text texture
                    text_surface = pygame.Surface(self.window_size, pygame.SRCALPHA)
                    font = pygame.font.SysFont("Mono", 10)
                    
                    # Render text to surface
                    # FPS counter
                    fps = clock.get_fps()
                    fps_text = f"FPS: {fps:.1f}"
                    fps_render = font.render(fps_text, True, (255, 255, 0))
                    text_surface.blit(fps_render, (100, 100))
                    
                    # Resolution display
                    res_text = f"Resolution: {self.fluid.resolution}x{self.fluid.resolution}"
                    res_render = font.render(res_text, True, (255, 255, 0))
                    text_surface.blit(res_render, (100, 140))
                    
                    # Controls text
                    controls_text = "Controls: R=Reset, Space=Pause, ESC=Exit"
                    controls_render = font.render(controls_text, True, (255, 255, 0))
                    text_surface.blit(controls_render, (100, self.window_size[1] - 130))
                    
                    # Convert text surface to OpenGL texture
                    text_data = pygame.image.tostring(text_surface, "RGBA", True)
                    text_texture = glGenTextures(1)
                    glBindTexture(GL_TEXTURE_2D, text_texture)
                    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
                    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
                    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.window_size[0], self.window_size[1], 0, 
                                GL_RGBA, GL_UNSIGNED_BYTE, text_data)
                    
                    # Setup for overlay rendering
                    glEnable(GL_TEXTURE_2D)
                    glEnable(GL_BLEND)
                    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
                    
                    # Render text overlay as a textured quad
                    glLoadIdentity()  # Reset modelview matrix
                    glBindTexture(GL_TEXTURE_2D, text_texture)
                    glBegin(GL_QUADS)
                    glTexCoord2f(0, 0); glVertex2f(0, 0)
                    glTexCoord2f(1, 0); glVertex2f(1, 0)
                    glTexCoord2f(1, 1); glVertex2f(1, 1)
                    glTexCoord2f(0, 1); glVertex2f(0, 1)
                    glEnd()
                except Exception as e:
                    print(f"Error rendering text: {e}")
                finally:
                    # Clean up texture
                    if text_texture is not None:
                        try:
                            glDeleteTextures([text_texture])
                        except Exception as e:
                            print(f"Error cleaning up text texture: {e}")
                
                # Update display
                pygame.display.flip()
                
                # Cap the frame rate
                clock.tick(120)
        
        except Exception as e:
            print(f"Error in main loop: {e}")
        
        finally:
            # Streamlined cleanup sequence - only do each step once
            print("Shutting down simulation...")
            
            # First ensure all OpenGL operations are complete
            glFinish()
            
            # Clean up the fluid simulation with our explicit method
            if hasattr(self.fluid, 'cleanup'):
                self.fluid.cleanup()
                
            # Finally close Pygame
            pygame.quit()
            print("Cleanup complete, exiting normally")


if __name__ == "__main__":
    # Parse command line arguments
    import argparse
    parser = argparse.ArgumentParser(description="Fluid Simulation")
    parser.add_argument("-r", "--resolution", type=int, default=128, help="Grid resolution (default: 128)")
    parser.add_argument("-f", "--fullscreen", action="store_true", help="Run in fullscreen mode")
    parser.add_argument("-w", "--windowed", action="store_true", help="Run in windowed mode")
    
    # Filter out Jupyter arguments if present
    import sys
    sys_args = [arg for arg in sys.argv[1:] if not arg.startswith('/') and not '.json' in arg]
    
    try:
        args = parser.parse_args(sys_args)
        
        # START FULLSCREEN BY DEFAULT unless windowed mode is explicitly requested
        use_fullscreen = not args.windowed
        
        # Set initial resolution (can be adjusted at runtime)
        app = FluidSimulationApp(initial_resolution=args.resolution, use_fullscreen=use_fullscreen)
        
        try:
            app.run()
        except Exception as e:
            print(f"Error in main loop: {e}")
            # Ensure cleanup happens even if there's an exception
            if hasattr(app, 'fluid'):
                app.fluid.cleanup()
            pygame.quit()
    except Exception as e:
        print(f"Error initializing application: {e}")
        pygame.quit()

# use fluid_dynamics2.cuf as kernel for fluid_dynamics.so (not fluid_dynamics3.cuf)

Added density at 0.68, 0.55
Added density at 0.43, 0.86
Added density at 0.36, 0.57
Added density at 0.87, 0.64
Added density at 0.25, 0.45
Shutting down simulation...
Fluid simulation resources cleaned up
Cleanup complete, exiting normally


CuPy version for comparison only on benchmark fps; 17fps vs 45-60fps means the simulation is not realistic

In [9]:
import numpy as np
import cupy as cp
import pygame
from pygame.locals import DOUBLEBUF, OPENGL
from OpenGL.GL import *
from OpenGL.GLU import *
import time
import sys
import os

class FluidSimulationCuPy:
    """High-performance fluid simulation using CuPy and OpenGL visualization."""
    
    def __init__(self, resolution=128):
        """Initialize the fluid simulation with the given resolution."""
        self.resolution = resolution
        self.window_size = (1280, 720)
        
        # Initialization flag for cleanup management
        self.initialized = True
        
        # Simulation parameters - EXTREME values for visible diffusion
        self.dt = 0.5           # Increased time step for faster diffusion
        self.diffusion = 0.25   # 5x higher for very prominent diffusion
        self.viscosity = 0.15   # 5x higher for strong velocity smoothing
        
        # Create arrays for fluid simulation
        self.density = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        self.density_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        
        self.vx = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        self.vy = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        self.vx_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        self.vy_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
        
        # Create buffer for getting density data
        self.density_buffer = np.zeros((self.resolution, self.resolution), dtype=np.float64)
        
        # Initialize texture ID to None - will be created in setup_opengl
        self.texture_id = None
        
        # Performance tracking
        self.fps = 0
        
        print(f"CuPy Fluid Simulation initialized with resolution {resolution}x{resolution}")
    
    def setup_opengl(self):
        """Set up OpenGL for rendering the fluid simulation."""
        # Set a dark background color
        glClearColor(0.1, 0.1, 0.1, 1.0)
        
        # Set up projection matrix
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluOrtho2D(0, 1, 0, 1)  # 2D orthographic projection
        
        # Set up modelview matrix
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Enable texturing and blending
        glEnable(GL_TEXTURE_2D)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        
        # Delete any existing texture first to prevent leaks
        if self.texture_id is not None:
            try:
                glDeleteTextures([self.texture_id])
            except:
                pass
        
        # Create texture for fluid display
        self.texture_id = glGenTextures(1)
        glBindTexture(GL_TEXTURE_2D, self.texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
        
        # Initialize with a blank texture
        blank_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
        blank_data[:,:,3] = 255  # Full alpha
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0,
                    GL_RGBA, GL_UNSIGNED_BYTE, blank_data)
        
        print(f"OpenGL initialized with texture ID: {self.texture_id}")
        
        # Check for OpenGL errors
        error = glGetError()
        if error != GL_NO_ERROR:
            print(f"OpenGL error during setup: {error}")
    
    def draw_test_pattern(self):
        """Draw a simple test pattern to verify OpenGL rendering."""
        if not self.initialized:
            return
            
        # Create a checkerboard pattern
        test_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
        
        # Make a more obvious pattern with multiple colors
        square_size = max(4, self.resolution // 16)
        for i in range(self.resolution):
            for j in range(self.resolution):
                pattern_type = ((i // square_size) + (j // square_size)) % 4
                
                if pattern_type == 0:
                    test_data[i, j, 0] = 255  # Red
                    test_data[i, j, 1] = 0
                    test_data[i, j, 2] = 0
                elif pattern_type == 1:
                    test_data[i, j, 0] = 0
                    test_data[i, j, 1] = 255  # Green
                    test_data[i, j, 2] = 0
                elif pattern_type == 2:
                    test_data[i, j, 0] = 0
                    test_data[i, j, 1] = 0
                    test_data[i, j, 2] = 255  # Blue
                else:
                    test_data[i, j, 0] = 255  # Yellow
                    test_data[i, j, 1] = 255
                    test_data[i, j, 2] = 0
                    
                test_data[i, j, 3] = 255  # Alpha
        
        # Ensure we have a valid texture ID
        if self.texture_id is None:
            self.texture_id = glGenTextures(1)
        
        # Bind and update the texture
        glBindTexture(GL_TEXTURE_2D, self.texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0, 
                    GL_RGBA, GL_UNSIGNED_BYTE, test_data)
        
        # Check for OpenGL errors
        error = glGetError()
        if error != GL_NO_ERROR:
            print(f"OpenGL error when creating test pattern: {error}")
        else:
            print("Test pattern texture created successfully")

    def update_texture(self):
        """Update the fluid texture with the current density field."""
        if not self.initialized:
            return
            
        try:
            # Copy density data from GPU to CPU
            self.density_buffer = cp.asnumpy(self.density)
            
            # Create texture data
            texture_data = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
            
            # Set a baseline dark blue background color
            texture_data[:,:,2] = 30  # Low blue background
            texture_data[:,:,3] = 255  # Full alpha
            
            # Normalize the density values
            # Use 95th percentile to avoid extreme outliers
            p95 = np.percentile(self.density_buffer, 95)
            norm_factor = 1.0 / max(p95, 0.1)
            
            # Map density values to colors
            for i in range(self.resolution):
                for j in range(self.resolution):
                    value = self.density_buffer[i, j] * norm_factor
                    value = min(value, 1.0)  # Clamp to [0,1]
                    
                    if value > 0.01:  # Threshold to avoid rendering very small values
                        # Color gradient from blue to cyan to yellow to red
                        if value < 0.25:
                            # Blue to cyan
                            ratio = value / 0.25
                            texture_data[i, j, 0] = 0  # No red
                            texture_data[i, j, 1] = int(255 * ratio)  # Increasing green
                            texture_data[i, j, 2] = 255  # Full blue
                        elif value < 0.5:
                            # Cyan to green
                            ratio = (value - 0.25) / 0.25
                            texture_data[i, j, 0] = 0  # No red
                            texture_data[i, j, 1] = 255  # Full green
                            texture_data[i, j, 2] = int(255 * (1 - ratio))  # Decreasing blue
                        elif value < 0.75:
                            # Green to yellow
                            ratio = (value - 0.5) / 0.25
                            texture_data[i, j, 0] = int(255 * ratio)  # Increasing red
                            texture_data[i, j, 1] = 255  # Full green
                            texture_data[i, j, 2] = 0  # No blue
                        else:
                            # Yellow to red
                            ratio = (value - 0.75) / 0.25
                            texture_data[i, j, 0] = 255  # Full red
                            texture_data[i, j, 1] = int(255 * (1 - ratio))  # Decreasing green
                            texture_data[i, j, 2] = 0  # No blue
            
            # Make sure we have a valid texture
            if self.texture_id is None:
                self.texture_id = glGenTextures(1)
                glBindTexture(GL_TEXTURE_2D, self.texture_id)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
                glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
            
            # Update the texture
            glBindTexture(GL_TEXTURE_2D, self.texture_id)
            glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.resolution, self.resolution, 0, 
                        GL_RGBA, GL_UNSIGNED_BYTE, texture_data)
        except Exception as e:
            print(f"Error updating texture: {e}")
    
    # FORCED DIFFUSION: Extremely aggressive diffusion for visible effect
    def diffuse(self, b, x, x0, diffusion, dt):
        """Diffuse density or velocity with maximum visibility.
        
        Parameters:
        - b: boundary condition type (0 for density, 1 for horizontal velocity, 2 for vertical velocity)
        - x: output array
        - x0: input array
        - diffusion: diffusion coefficient
        - dt: time step
        """
        # Much higher diffusion scaling for very visible effects
        a = dt * diffusion * self.resolution * 2.0
        
        # Initialize with input values
        x[:] = x0[:]
        
        # Increased iterations for better spreading
        for k in range(20):
            # More aggressive diffusion formula - heavily weights the neighbors
            x[1:-1, 1:-1] = (x0[1:-1, 1:-1] * 0.5 + a * (
                x[2:, 1:-1] + x[:-2, 1:-1] + 
                x[1:-1, 2:] + x[1:-1, :-2]
            )) / (0.5 + 4 * a)  # Reduced center weight for stronger diffusion
            
            # Apply boundary conditions
            self.set_boundary(b, x)
            
        # Apply a second blur pass for density to ensure visible spreading
        if b == 0:  # Only for density
            for _ in range(2):  # Multiple passes for stronger effect
                temp = x.copy()
                x[1:-1, 1:-1] = (
                    temp[1:-1, 1:-1] * 0.6 +
                    temp[2:, 1:-1] * 0.1 +
                    temp[:-2, 1:-1] * 0.1 +
                    temp[1:-1, 2:] * 0.1 +
                    temp[1:-1, :-2] * 0.1
                )
                self.set_boundary(b, x)
    
    def project(self, vx, vy, p, div):
        """Project the velocity field to be incompressible."""
        # Calculate divergence
        div[1:-1, 1:-1] = -0.5 * (
            vx[2:, 1:-1] - vx[:-2, 1:-1] + 
            vy[1:-1, 2:] - vy[1:-1, :-2]
        ) / self.resolution
        
        p.fill(0)
        self.set_boundary(0, div)
        self.set_boundary(0, p)
        
        # Jacobi iterations for pressure
        for k in range(20):  # Same as SOLVER_ITERATIONS in the original code
            p[1:-1, 1:-1] = (div[1:-1, 1:-1] + p[2:, 1:-1] + p[:-2, 1:-1] + p[1:-1, 2:] + p[1:-1, :-2]) / 4
            self.set_boundary(0, p)
        
        # Subtract pressure gradient from velocity
        vx[1:-1, 1:-1] -= 0.5 * (p[2:, 1:-1] - p[:-2, 1:-1]) * self.resolution
        vy[1:-1, 1:-1] -= 0.5 * (p[1:-1, 2:] - p[1:-1, :-2]) * self.resolution
        
        self.set_boundary(1, vx)
        self.set_boundary(2, vy)
    
    # Enhanced advection for better fluid movement
    def advect(self, b, d, d0, vx, vy, dt):
        """Advect density or velocity with enhanced spreading."""
        dt0 = dt * self.resolution
        
        # Create coordinate arrays for lookups
        i_range = cp.arange(1, self.resolution - 1)
        j_range = cp.arange(1, self.resolution - 1)
        
        # Create meshgrid for vectorized operations
        jj, ii = cp.meshgrid(j_range, i_range)
        
        # Calculate positions to sample from with slightly reduced velocity
        # This creates more spreading as particles move slower
        x = ii - dt0 * vx[1:-1, 1:-1] * 0.9  # Reduced velocity for more diffusion
        y = jj - dt0 * vy[1:-1, 1:-1] * 0.9
        
        # Clamp positions to valid range with a small margin for interpolation
        x = cp.clip(x, 0.5, self.resolution - 1.5)
        y = cp.clip(y, 0.5, self.resolution - 1.5)
        
        # Get integer positions for bilinear interpolation
        i0 = cp.floor(x).astype(cp.int32)
        j0 = cp.floor(y).astype(cp.int32)
        i1 = i0 + 1
        j1 = j0 + 1
        
        # Get fractional parts for interpolation weights
        s1 = x - i0
        s0 = 1 - s1
        t1 = y - j0
        t0 = 1 - t1
        
        # Ensure indices are within bounds
        i0 = cp.clip(i0, 0, self.resolution - 1)
        j0 = cp.clip(j0, 0, self.resolution - 1)
        i1 = cp.clip(i1, 0, self.resolution - 1)
        j1 = cp.clip(j1, 0, self.resolution - 1)
        
        # Modified interpolation with slight diffusion boost
        d[1:-1, 1:-1] = (
            s0 * (t0 * d0[i0, j0] + t1 * d0[i0, j1]) +
            s1 * (t0 * d0[i1, j0] + t1 * d0[i1, j1])
        )
        
        # Apply boundary conditions
        self.set_boundary(b, d)
        
        # For density, apply a very slight blur during advection
        if b == 0:
            temp = d.copy()
            d[1:-1, 1:-1] = (
                temp[1:-1, 1:-1] * 0.9 +
                temp[2:, 1:-1] * 0.025 +
                temp[:-2, 1:-1] * 0.025 +
                temp[1:-1, 2:] * 0.025 +
                temp[1:-1, :-2] * 0.025
            )
            self.set_boundary(b, d)
    
    def set_boundary(self, b, x):
        """Set boundary conditions."""
        # Vertical edges
        if b == 1:  # Horizontal velocity component
            x[0, :] = -x[1, :]
            x[-1, :] = -x[-2, :]
        else:
            x[0, :] = x[1, :]
            x[-1, :] = x[-2, :]
            
        # Horizontal edges
        if b == 2:  # Vertical velocity component
            x[:, 0] = -x[:, 1]
            x[:, -1] = -x[:, -2]
        else:
            x[:, 0] = x[:, 1]
            x[:, -1] = x[:, -2]
            
        # Corners - average of neighboring cells
        x[0, 0] = 0.5 * (x[1, 0] + x[0, 1])
        x[0, -1] = 0.5 * (x[1, -1] + x[0, -2])
        x[-1, 0] = 0.5 * (x[-2, 0] + x[-1, 1])
        x[-1, -1] = 0.5 * (x[-2, -1] + x[-1, -2])
    
    # EXTREME STEP: Apply additional diffusion every step
    def step(self):
        """Advance the simulation by one time step with forced diffusion."""
        if not self.initialized:
            return
            
        # Apply super high diffusion rates - 3x base values
        diffusion_rate = self.diffusion * 3.0
        viscosity_rate = self.viscosity * 2.0
        
        # Velocity diffusion (viscosity effect)
        self.diffuse(1, self.vx_prev, self.vx, viscosity_rate, self.dt)
        self.diffuse(2, self.vy_prev, self.vy, viscosity_rate, self.dt)
        
        # Project to enforce incompressibility
        self.project(self.vx_prev, self.vy_prev, self.vx, self.vy)
        
        # Swap velocity arrays for advection step
        self.vx, self.vx_prev = self.vx_prev, self.vx
        self.vy, self.vy_prev = self.vy_prev, self.vy
        
        # Advect velocity
        self.advect(1, self.vx, self.vx_prev, self.vx_prev, self.vy_prev, self.dt)
        self.advect(2, self.vy, self.vy_prev, self.vx_prev, self.vy_prev, self.dt)
        
        # Project again
        self.project(self.vx, self.vy, self.vx_prev, self.vy_prev)
        
        # Density diffusion - very high rate
        self.diffuse(0, self.density_prev, self.density, diffusion_rate, self.dt)
        
        # Swap density arrays for advection step
        self.density, self.density_prev = self.density_prev, self.density
        
        # Advect density
        self.advect(0, self.density, self.density_prev, self.vx, self.vy, self.dt)
        
        # After advection, apply an additional forced diffusion for more spreading
        temp = self.density.copy()
        self.density[1:-1, 1:-1] = (
            temp[1:-1, 1:-1] * 0.8 +
            temp[2:, 1:-1] * 0.05 +
            temp[:-2, 1:-1] * 0.05 +
            temp[1:-1, 2:] * 0.05 +
            temp[1:-1, :-2] * 0.05
        )
        self.set_boundary(0, self.density)
    
    def get_density(self):
        """Get the current density field."""
        if not self.initialized:
            return self.density_buffer
        
        self.density_buffer = cp.asnumpy(self.density)
        return self.density_buffer
    
    # EXTREME DENSITY: Much stronger density addition for visible diffusion
    def add_density(self, x, y, amount=1.0):
        """Add density at the specified position with forced spreading effect."""
        if not self.initialized:
            return
            
        try:
            # Convert from normalized [0,1] coordinates to grid coordinates
            grid_x = int(x * self.resolution)
            grid_y = int(y * self.resolution)
            
            # Clamp to valid range
            grid_x = max(1, min(grid_x, self.resolution-2))
            grid_y = max(1, min(grid_y, self.resolution-2))
            
            # MUCH higher density values - 4x the tensor core amount for center
            center_amount = amount * 400.0
            self.density[grid_x, grid_y] += center_amount
            
            # Create a larger stencil with graduated values for better visibility
            # Inner ring (4 adjacent cells) - higher than tensor core
            radius1 = 1
            for i in range(-radius1, radius1+1):
                for j in range(-radius1, radius1+1):
                    if abs(i) + abs(j) == 1:  # Manhattan distance = 1 (4 adjacent cells)
                        nx, ny = grid_x+i, grid_y+j
                        if 0 < nx < self.resolution-1 and 0 < ny < self.resolution-1:
                            self.density[nx, ny] += center_amount * 0.5  # 50% of center
            
            # Middle ring - new addition for better spreading
            radius2 = 2
            for i in range(-radius2, radius2+1):
                for j in range(-radius2, radius2+1):
                    manhattan = abs(i) + abs(j)
                    if manhattan > 1 and manhattan <= 2:
                        nx, ny = grid_x+i, grid_y+j
                        if 0 < nx < self.resolution-1 and 0 < ny < self.resolution-1:
                            self.density[nx, ny] += center_amount * 0.25  # 25% of center
            
            # Outer ring - additional light spreading
            radius3 = 3
            for i in range(-radius3, radius3+1):
                for j in range(-radius3, radius3+1):
                    manhattan = abs(i) + abs(j)
                    if manhattan > 2 and manhattan <= 3:
                        nx, ny = grid_x+i, grid_y+j
                        if 0 < nx < self.resolution-1 and 0 < ny < self.resolution-1:
                            self.density[nx, ny] += center_amount * 0.1  # 10% of center
        except Exception as e:
            print(f"Error adding density: {e}")
        except Exception as e:
            print(f"Error adding density: {e}")
    
    # Enhanced velocity for more dynamic movement
    def add_velocity(self, x, y, vx, vy):
        """Add velocity with wider spread for better effect."""
        if not self.initialized:
            return
            
        try:
            # Convert from normalized [0,1] coordinates to grid coordinates
            grid_x = int(x * self.resolution)
            grid_y = int(y * self.resolution)
            
            # Clamp to valid range
            grid_x = max(1, min(grid_x, self.resolution-2))
            grid_y = max(1, min(grid_y, self.resolution-2))
            
            # Much stronger velocity scaling
            vx_scaled = vx * 40.0  # 4x tensor core version
            vy_scaled = vy * 40.0  # 4x tensor core version
            
            # Add velocity to center and surrounding cells
            radius = 2
            for i in range(-radius, radius+1):
                for j in range(-radius, radius+1):
                    nx, ny = grid_x+i, grid_y+j
                    if 0 < nx < self.resolution-1 and 0 < ny < self.resolution-1:
                        # Distance-based falloff
                        dist = np.sqrt(i*i + j*j)
                        if dist <= radius:
                            # Stronger at center, weaker at edges
                            factor = 1.0 - (dist / (radius + 0.5))
                            self.vx[nx, ny] += vx_scaled * factor
                            self.vy[nx, ny] += vy_scaled * factor
        except Exception as e:
            print(f"Error adding velocity: {e}")
        except Exception as e:
            print(f"Error adding velocity: {e}")
    
    def reset(self):
        """Reset the simulation to its initial state."""
        if self.initialized:
            try:
                # Reset all arrays
                self.density.fill(0)
                self.density_prev.fill(0)
                self.vx.fill(0)
                self.vy.fill(0)
                self.vx_prev.fill(0)
                self.vy_prev.fill(0)
                self.density_buffer.fill(0)
                print("Simulation reset")
            except Exception as e:
                print(f"Error resetting fluid simulation: {e}")
    
    def change_resolution(self, new_resolution):
        """Change the simulation resolution."""
        if not self.initialized or new_resolution == self.resolution:
            return
            
        try:
            # Store current state before resizing
            self.initialized = False
            
            # Update resolution
            self.resolution = new_resolution
            
            # Create new arrays with updated size
            self.density = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            self.density_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            self.vx = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            self.vy = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            self.vx_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            self.vy_prev = cp.zeros((self.resolution, self.resolution), dtype=cp.float64)
            
            # Reset density buffer
            self.density_buffer = np.zeros((self.resolution, self.resolution), dtype=np.float64)
            
            # Reset texture ID so it will be recreated with the new resolution
            if self.texture_id is not None:
                try:
                    glDeleteTextures([self.texture_id])
                except:
                    pass
                self.texture_id = None
            
            # Re-initialize
            self.initialized = True
            
            print(f"Resolution changed to {self.resolution}x{self.resolution}")
        except Exception as e:
            print(f"Error changing resolution: {e}")
    
    def cleanup(self):
        """Explicit cleanup method to be called when shutting down."""
        if self.initialized:
            try:
                # First clean up OpenGL resources
                if self.texture_id is not None:
                    try:
                        glDeleteTextures([self.texture_id])
                        self.texture_id = None
                    except Exception as e:
                        print(f"Error deleting texture: {e}")
                
                # Free GPU memory
                del self.density
                del self.density_prev
                del self.vx
                del self.vy
                del self.vx_prev
                del self.vy_prev
                
                # Clear memory pools
                cp.get_default_memory_pool().free_all_blocks()
                cp.get_default_pinned_memory_pool().free_all_blocks()
                
                self.initialized = False
                print("Fluid simulation resources cleaned up")
            except Exception as e:
                print(f"Error during cleanup: {e}")
                self.initialized = False
    
    def __del__(self):
        """Clean up resources when the object is destroyed."""
        self.cleanup()


def run_in_jupyter(resolution=128):
    """Run the fluid simulation in a Jupyter notebook."""
    app = FluidSimulationApp(initial_resolution=resolution, use_fullscreen=False)
    return app


class FluidSimulationApp:
    """Main application for the fluid simulation."""
    
    def __init__(self, initial_resolution=128, use_fullscreen=True):
        """Initialize the application with the given resolution."""
        # Initialize PyGame
        pygame.init()
        
        # Set default window size for all modes
        self.window_size = (1024, 768)
        
        # Create display with desired mode
        if use_fullscreen:
            display_info = pygame.display.Info()
            self.window_size = (display_info.current_w, display_info.current_h)
            self.display = pygame.display.set_mode(self.window_size, DOUBLEBUF | OPENGL | pygame.FULLSCREEN)
            self.fullscreen = True
        else:
            self.display = pygame.display.set_mode(self.window_size, DOUBLEBUF | OPENGL)
            self.fullscreen = False
        
        # Get actual window size
        self.window_size = pygame.display.get_surface().get_size()
        pygame.display.set_caption("Fluid Dynamics Simulation (CuPy)")
        
        print(f"Window initialized: size={self.window_size}, fullscreen={self.fullscreen}")
        
        # Store resolution
        self.resolution = initial_resolution
        
        # Create the fluid simulation - Initialize only once!
        self.fluid = FluidSimulationCuPy(initial_resolution)
        
        # Set up OpenGL
        self.fluid.setup_opengl()
        
        # Make sure our viewport is set up correctly
        self.reset_opengl_viewport()
        
        # Mouse tracking
        self.prev_mouse_pos = None
        self.mouse_down = False
        
        # Control flags
        self.paused = False
        
        print("Controls: R=Reset, Space=Pause, ESC=Exit")
    
    def reset_opengl_viewport(self):
        """Reset the OpenGL viewport and projection to fit the current window."""
        # Set viewport to cover the whole window
        glViewport(0, 0, self.window_size[0], self.window_size[1])
        
        # Reset projection matrix for 2D rendering
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluOrtho2D(0, 1, 0, 1)  # Map to [0,1] coordinate space
        
        # Reset modelview matrix
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Make sure texturing and blending are enabled
        glEnable(GL_TEXTURE_2D)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
    
    def toggle_fullscreen(self):
        """Toggle fullscreen mode."""
        # Toggle fullscreen state
        self.fullscreen = not self.fullscreen
        
        # Save current resolution before switching
        current_resolution = self.fluid.resolution
        
        # Properly clean up current OpenGL context and resources
        glFinish()  # Make sure all OpenGL commands are done
        
        # Clean up texture resources
        if self.fluid.texture_id is not None:
            try:
                glDeleteTextures([self.fluid.texture_id])
                self.fluid.texture_id = None
            except Exception as e:
                print(f"Error cleaning up texture: {e}")
        
        # Create new display with desired mode
        if self.fullscreen:
            # Get display info for native resolution
            display_info = pygame.display.Info()
            new_size = (display_info.current_w, display_info.current_h)
            self.display = pygame.display.set_mode(new_size, DOUBLEBUF | OPENGL | pygame.FULLSCREEN)
        else:
            # Return to windowed mode with standard size
            self.display = pygame.display.set_mode((1024, 768), DOUBLEBUF | OPENGL)
        
        # Update window size
        self.window_size = pygame.display.get_surface().get_size()
        print(f"Toggled to {'fullscreen' if self.fullscreen else 'windowed'} mode, size: {self.window_size}")
                
        # Recreate the fluid texture
        self.fluid.setup_opengl()
        
        # Reset viewport
        self.reset_opengl_viewport()
    
    def draw_fluid_texture(self):
        """Draw the fluid simulation texture to fill the entire screen."""
        # Ensure we have a valid texture
        if self.fluid.texture_id is None:
            return
            
        try:
            # Set up for drawing
            glMatrixMode(GL_MODELVIEW)
            glLoadIdentity()
            
            # Bind the fluid simulation texture
            glEnable(GL_TEXTURE_2D)
            glBindTexture(GL_TEXTURE_2D, self.fluid.texture_id)
            
            # Draw a quad that covers the entire screen in the [0,1] coordinate space
            glColor4f(1.0, 1.0, 1.0, 1.0)  # Full color/alpha
            glBegin(GL_QUADS)
            # These coordinates map to the entire viewport in the [0,1] space
            glTexCoord2f(0.0, 0.0); glVertex2f(0.0, 0.0)  # Bottom-left
            glTexCoord2f(1.0, 0.0); glVertex2f(1.0, 0.0)  # Bottom-right
            glTexCoord2f(1.0, 1.0); glVertex2f(1.0, 1.0)  # Top-right
            glTexCoord2f(0.0, 1.0); glVertex2f(0.0, 1.0)  # Top-left
            glEnd()
            
            # Disable texturing when done
            glDisable(GL_TEXTURE_2D)
        except Exception as e:
            print(f"Error drawing fluid texture: {e}")
    
    def run(self):
        """Run the main application loop."""
        running = True
        clock = pygame.time.Clock()
        
        # Visualization mode - start with simulation
        show_simulation = True
        
        print("Starting fluid simulation. Press ESC to exit.")
        
        try:
            while running:
                # Handle events
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                        
                    elif event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            running = False
                        elif event.key == pygame.K_r:
                            self.fluid.reset()
                            print("Simulation reset")
                        elif event.key == pygame.K_SPACE:
                            self.paused = not self.paused
                            print(f"Simulation {'paused' if self.paused else 'resumed'}")
                        elif event.key == pygame.K_t:
                            # Toggle between test pattern and simulation
                            show_simulation = not show_simulation
                            if not show_simulation:
                                self.fluid.draw_test_pattern()
                                print("Switched to test pattern")
                            else:
                                print("Switched to simulation")
                        elif event.key == pygame.K_UP:
                            # Increase resolution
                            new_res = min(self.fluid.resolution * 2, 512)
                            self.fluid.change_resolution(new_res)
                        elif event.key == pygame.K_DOWN:
                            # Decrease resolution
                            new_res = max(self.fluid.resolution // 2, 32)
                            self.fluid.change_resolution(new_res)
                            
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        if event.button == 1:  # Left mouse button
                            self.mouse_down = True
                            self.prev_mouse_pos = pygame.mouse.get_pos()
                            
                            # Add initial density at the exact mouse position
                            if show_simulation:
                                x, y = self.prev_mouse_pos
                                
                                # Fix the coordinate mapping:
                                # Since we're seeing up→right and clockwise→counter-clockwise,
                                # we need to swap x and y AND flip one of them
                                # Try this mapping: mouse_x → fluid_y, mouse_y → (1-fluid_x)
                                nx = 1.0 - y / self.window_size[1]  # Inverted y becomes x
                                ny = x / self.window_size[0]        # x becomes y
                                
                                self.fluid.add_density(nx, ny, 2.0)
                                print(f"Added density at {nx:.2f}, {ny:.2f}")
                            
                    elif event.type == pygame.MOUSEBUTTONUP:
                        if event.button == 1:  # Left mouse button
                            self.mouse_down = False
                            self.prev_mouse_pos = None
                
                # Handle mouse motion
                if self.mouse_down and show_simulation:
                    x, y = pygame.mouse.get_pos()
                    
                    # Fix the coordinate mapping (same as in mouse click handling)
                    nx = 1.0 - y / self.window_size[1]  # Inverted y becomes x
                    ny = x / self.window_size[0]        # x becomes y
                    
                    self.fluid.add_density(nx, ny, 1.0)
                    
                    # Add velocity if mouse has moved
                    if self.prev_mouse_pos is not None:
                        prev_x, prev_y = self.prev_mouse_pos
                        
                        # Calculate velocity with the same coordinate transformation
                        dx = x - prev_x
                        dy = y - prev_y
                        
                        # Transform the velocity vector to match our coordinate transformation
                        vx = -dy / self.window_size[1] * 5.0  # negative dy becomes vx
                        vy = dx / self.window_size[0] * 5.0   # dx becomes vy
                        
                        # Add velocity
                        self.fluid.add_velocity(nx, ny, vx, vy)
                    
                    # Update previous position
                    self.prev_mouse_pos = (x, y)
                
                # Step the simulation if not paused and showing simulation
                if not self.paused and show_simulation:
                    self.fluid.step()
                    self.fluid.update_texture()
                
                # Clear the screen
                glClear(GL_COLOR_BUFFER_BIT)
                
                # Draw the current texture (test pattern or simulation)
                self.draw_fluid_texture()
                
                # Text rendering with proper cleanup
                text_texture = None
                try:
                    # Create and setup text texture
                    text_surface = pygame.Surface(self.window_size, pygame.SRCALPHA)
                    font = pygame.font.SysFont("Mono", 10)
                    
                    # Render text to surface
                    # FPS counter
                    fps = clock.get_fps()
                    fps_text = f"FPS: {fps:.1f}"
                    fps_render = font.render(fps_text, True, (255, 255, 0))
                    text_surface.blit(fps_render, (10, 10))
                    
                    # Resolution display
                    res_text = f"Resolution: {self.fluid.resolution}x{self.fluid.resolution}"
                    res_render = font.render(res_text, True, (255, 255, 0))
                    text_surface.blit(res_render, (10, 40))
                    
                    # Controls text
                    controls_text = "Controls: R=Reset, Space=Pause, ESC=Exit"
                    controls_render = font.render(controls_text, True, (255, 255, 0))
                    text_surface.blit(controls_render, (10, self.window_size[1] - 30))
                    
                    # Convert text surface to OpenGL texture
                    text_data = pygame.image.tostring(text_surface, "RGBA", True)
                    text_texture = glGenTextures(1)
                    glBindTexture(GL_TEXTURE_2D, text_texture)
                    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
                    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
                    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, self.window_size[0], self.window_size[1], 0, 
                                GL_RGBA, GL_UNSIGNED_BYTE, text_data)
                    
                    # Setup for overlay rendering
                    glEnable(GL_TEXTURE_2D)
                    glEnable(GL_BLEND)
                    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
                    
                    # Render text overlay as a textured quad
                    glLoadIdentity()  # Reset modelview matrix
                    glBindTexture(GL_TEXTURE_2D, text_texture)
                    glBegin(GL_QUADS)
                    glTexCoord2f(0, 0); glVertex2f(0, 0)
                    glTexCoord2f(1, 0); glVertex2f(1, 0)
                    glTexCoord2f(1, 1); glVertex2f(1, 1)
                    glTexCoord2f(0, 1); glVertex2f(0, 1)
                    glEnd()
                except Exception as e:
                    print(f"Error rendering text: {e}")
                finally:
                    # Clean up texture
                    if text_texture is not None:
                        try:
                            glDeleteTextures([text_texture])
                        except Exception as e:
                            print(f"Error cleaning up text texture: {e}")
                
                # Update display
                pygame.display.flip()
                
                # Cap the frame rate
                clock.tick(120)
        
        except Exception as e:
            print(f"Error in main loop: {e}")
        
        finally:
            # Streamlined cleanup sequence - only do each step once
            print("Shutting down simulation...")
            
            # First ensure all OpenGL operations are complete
            glFinish()
            
            # Clean up the fluid simulation with our explicit method
            if hasattr(self.fluid, 'cleanup'):
                self.fluid.cleanup()
                
            # Finally close Pygame
            pygame.quit()
            print("Cleanup complete, exiting normally")


if __name__ == "__main__":
    # Parse command line arguments
    import argparse
    parser = argparse.ArgumentParser(description="Fluid Simulation with CuPy")
    parser.add_argument("-r", "--resolution", type=int, default=128, help="Grid resolution (default: 128)")
    parser.add_argument("-f", "--fullscreen", action="store_true", help="Run in fullscreen mode")
    parser.add_argument("-w", "--windowed", action="store_true", help="Run in windowed mode")
    
    # Filter out Jupyter arguments if present
    import sys
    sys_args = [arg for arg in sys.argv[1:] if not arg.startswith('/') and not '.json' in arg]
    
    try:
        args = parser.parse_args(sys_args)
        
        # START FULLSCREEN BY DEFAULT unless windowed mode is explicitly requested
        use_fullscreen = not args.windowed
        
        # Set initial resolution (can be adjusted at runtime)
        app = FluidSimulationApp(initial_resolution=args.resolution, use_fullscreen=use_fullscreen)
        
        try:
            app.run()
        except Exception as e:
            print(f"Error in main loop: {e}")
            # Ensure cleanup happens even if there's an exception
            if hasattr(app, 'fluid'):
                app.fluid.cleanup()
            pygame.quit()
    except Exception as e:
        print(f"Error initializing application: {e}")
        pygame.quit()

Added density at 0.67, 0.52
Added density at 0.70, 0.38
Added density at 0.45, 0.45
Added density at 0.73, 0.48
Shutting down simulation...
Fluid simulation resources cleaned up
Cleanup complete, exiting normally
